# LoCoMock (LogP-corrected Membrane Docking)

This program is a modified extension of Jupyter_Dock[https://github.com/AngelRuizMoreno/Jupyter_Dock].

Place this notebook on the "Jupyter_Dock" directory.

In [ ]:
from pymol import cmd
import py3Dmol

from vina import Vina

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdCoordGen
from rdkit.ML.Cluster import Butina

import sys, os
import shutil
sys.path.insert(1, 'utilities/')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf

import glob

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

## Plot Settings

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['font.family'] = "Helvetica"

matplotlib.rcParams.update({'figure.figsize': [83/25.4, 83/25.4]})

## Project Settings

In [ ]:
str_id = "targetID"
n_poses = 1000
membrane_pos = 14.7
buffer = 4.7

In [ ]:
os.makedirs(f"LoCoMock/LoCoMock_{str_id}/", exist_ok=True)
os.chdir(f"LoCoMock/LoCoMock_{str_id}/")

# Prepare input

Please predict the orientation and membrane position for your protein at PPM server [https://opm.phar.umich.edu/ppm_server3].


Place PDB files for protein (protein.pdb) and ligand (ligand.pdb) in the ./LoCoMock/{str_id} .


The protein.pdb should contain the thickness of the membrane in the first line otherwise the default value 14.7 is used.


Ex. "REMARK      1/2 of bilayer thickness:   17.4"

In [ ]:
protein_pdb = ""
with open(f"./protein.pdb", "r") as f:
    resn = None
    chain_id = None
    for line in f:
        if "bilayer" in line:
            membrane_pos = float(line.split()[5])
        if "ATOM" in line and line[21]:
            protein_pdb += line
print(f"The membrane position is {membrane_pos}")

In [ ]:
with open(f"./protein_clean.pdb", "w") as f:
    f.write(protein_pdb)

## Prepare Protein

In [ ]:
!../../bin/lepro_linux_x86 {'protein_clean.pdb'}
os.rename('pro.pdb','protein_H.pdb')

## Prepare Ligand

In [ ]:
sdf_mol = pybel.readfile("pdb", f"./ligand.pdb")
for index,mol in enumerate(sdf_mol):
    out=pybel.Outputfile(filename='./ligand.pdbqt', format='pdbqt',overwrite=True, opt={"h": 7.0})
    mol.OBMol.AddHydrogens()
    out.write(mol)
    out.close()
    
    out=pybel.Outputfile(filename='./ligand_clean.pdb', format='pdb',overwrite=True, opt={"h": 7.0})
    out.write(mol)
    out.close()

In [ ]:
mol_noH = Chem.MolFromPDBFile("./ligand_clean.pdb")
mol = mol_noH
rdCoordGen.AddCoords(mol)
Draw.MolToImage(mol)

In [ ]:
from rdkit.Chem import Descriptors
print(f"The logP of the target ligand is {Descriptors.MolLogP(mol)}")

In [ ]:
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import rdMolDescriptors
contribs = rdMolDescriptors._CalcCrippenContribs(mol)
fig = SimilarityMaps.GetSimilarityMapFromWeights(mol,[x for x,y in contribs], colorMap='seismic', contourLines=0, sigma=0.02, size=(500,500))
fig.savefig('ligand_logP.png',bbox_inches='tight', dpi=300) 
plt.show()

## Input System Visualization

In [ ]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('protein_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open(f"./ligand.pdb",'r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

# Docking with AutoDock Vina

In [ ]:
!../../bin/prepare_receptor -v -r protein_H.pdb -o protein_H.pdbqt

In [ ]:
cmd.load(filename='protein_H.pdb',format='pdb',object='prot')
cmd.load(filename='ligand.pdb',format='pdb',object='lig')

center, size= getbox(selection='prot',extending=5.0,software='vina')

#If you want to specify the docking area
#center = {'center_x': 7.77, 'center_y': 7.77, 'center_z': 7.77}
#size = {'size_x': 77.7, 'size_y': 77.7, 'size_z': 77.7}

cmd.delete('all')

print(center,'\n',size)

## Docking

In [ ]:
v = Vina(sf_name='vina')

In [ ]:
v.set_receptor('protein_H.pdbqt')

In [ ]:
v.set_ligand_from_file('ligand.pdbqt')

In [ ]:
v.compute_vina_maps(center=[center['center_x'], center['center_y'], center['center_z']], 
                    box_size=[size['size_x'], size['size_y'], size['size_z']])

In [ ]:
# Dock the ligand
v.dock(exhaustiveness=10, n_poses=n_poses, min_rmsd=1)

In [ ]:
v.write_poses('ligand_vina_out.pdbqt', n_poses=n_poses, overwrite=True, energy_range=500)
vina_score = v.energies(n_poses=n_poses, energy_range=500)[:, 0]
np.savetxt("vina_energy.txt", vina_score)

In [ ]:
pdbqt_to_sdf(pdbqt_file='ligand_vina_out.pdbqt',output='ligand_vina_out.sdf')

In [ ]:
os.makedirs("./result_pdb", exist_ok=True)

In [ ]:
results = [m for m in pybel.readfile(filename='ligand_vina_out.pdbqt',format='pdbqt')]

for index, pose in enumerate(results):
    out=pybel.Outputfile(filename=f'./result_pdb/ligand_{index}.pdb',format='pdb',overwrite=True)
    out.write(pose)
out.close()

In [ ]:
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import rdMolDescriptors


In [ ]:
fig = plt.figure(figsize=(83/25.4, 60/25.4))

plt.hist(vina_score, bins=20, color="#EEEEEE", edgecolor="black", linewidth=0.5)
plt.xlabel("Docking score (kcal/mol)")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("Docking_score.pdf", dpi=300)
plt.show()

# LoCoMock scoring

In [ ]:
#Kucerka et al., 2008 Biophys. J. 95: 2356-2367
#https://dx.doi.org/10.1529%2Fbiophysj.108.132662
import math
mol_logp = sum([x[0] for x in contribs])
WH_scores = []
for pdb_file in sorted(glob.glob('./result_pdb/ligand_*.pdb'), key = lambda x: int(x.replace("./result_pdb/ligand_","").replace(".pdb",""))):
    pose = Chem.MolFromPDBFile(pdb_file)
    coordinates = [a for a in pose.GetConformers()[0].GetPositions()]
    WH_score = []
    for coor, cont in zip(coordinates, contribs):
        if coor[2] <0 :
            sigmoid = -(1/ (1 + math.e**-( coor[2]+membrane_pos)))
            WH_score.append(sigmoid *cont[0])
        elif coor[2] >= 0 :
            sigmoid = -(1/ (1 + math.e**( coor[2]-membrane_pos)))
            WH_score.append(sigmoid *cont[0])
    WH_scores.append(sum(WH_score))
    
fig = plt.figure(figsize=(83/25.4, 60/25.4))

plt.hist(logp_scores, bins=20, color="#EEEEEE", edgecolor="black", linewidth=0.5)
plt.xlabel("WH score")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("WH_score.pdf", dpi=300)

plt.show()
#[print(i, s) for i,s in enumerate(scores)]

np.savetxt("WH_score.txt", WH_scores)

In [ ]:
multiple_score = [v+(2.303*(8.314*300)*(logp)/4.184/1000)  for v, logp in zip(vina_score, logp_scores)]
fig = plt.figure(figsize=(83/25.4, 60/25.4))
plt.hist(multiple_score, bins=20, color="#EEEEEE", edgecolor="black", linewidth=0.5)
plt.xlabel("LoCoMock score")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("LoCoMock_score.pdf", dpi=300)

plt.show()

np.savetxt("locomock_score.txt", multiple_score)

In [ ]:
WH_scores = np.loadtxt("WH_score.txt")

In [ ]:
vina_score = np.loadtxt("vina_energy.txt")

In [ ]:
multiple_score = np.loadtxt("locomock_score.txt")

In [ ]:
num_best = 10

# Best poses by LoCoMock

In [ ]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('protein_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

for i in sorted(enumerate(multiple_score), key=lambda x: x[1], reverse=False)[:num_best]:
    view.addModel(open(f"./result_pdb/ligand_{i[0]}.pdb",'r').read(),format='mol2')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

# Best poses by Vina

In [ ]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('protein_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

for i in sorted(enumerate(vina_score), key=lambda x: x[1])[:num_best]:
    view.addModel(open(f"./result_pdb/ligand_{i[0]}.pdb",'r').read(),format='mol2')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

# Export models

In [ ]:
file_list = ""
for i, s in enumerate(multiple_score):
    file_list += f" ./result_pdb/ligand_{i}.pdb"
!cat $file_list | sed -e "s/END/ENDMDL/g" > ./ligand_all.pdb

file_list = ""
for i in sorted(enumerate(multiple_score), key=lambda x: x[1], reverse=False)[:10]:
    file_list += f" ./result_pdb/ligand_{i[0]}.pdb"
!cat $file_list | sed -e "s/END/ENDMDL/g" > ./ligand_best.pdb

file_list = ""
for i in sorted(enumerate(multiple_score), key=lambda x: x[1], reverse=False)[-10:]:
    file_list += f" ./result_pdb/ligand_{i[0]}.pdb"
!cat $file_list | sed -e "s/END/ENDMDL/g" > ./ligand_worst.pdb

In [ ]:
file_list = ""
for i in sorted(enumerate(multiple_score), key=lambda x: x[1], reverse=False)[:num_best]:
    file_list += f" ./result_pdb/ligand_{i[0]}.pdb"
!cat $file_list | sed -e "s/END/ENDMDL/g" > ./ligand_best_LoCoMock.pdb

file_list = ""
best_100_list = []
for i in sorted(enumerate(vina_score), key=lambda x: x[1], reverse=False)[:num_best]:
    file_list += f" ./result_pdb/ligand_{i[0]}.pdb"
!cat $file_list | sed -e "s/END/ENDMDL/g" > ./ligand_best_vina.pdb